<a href="https://colab.research.google.com/github/chunbo777/dacon_prac/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!git clone https://github.com/chunbo777/deep-learning-from-scratch

fatal: destination path 'deep-learning-from-scratch' already exists and is not an empty directory.


In [ ]:
import sys
sys.path.insert(0, "/content/deep-learning-from-scratch")
from common.util import im2col



In [ ]:
import numpy as np

In [ ]:
x1= np.random.rand(1,3,7,7)

In [ ]:
x1

array([[[[0.78876353, 0.92261197, 0.50132567, 0.6058759 , 0.78779392,
          0.06958523, 0.60460621],
         [0.01936705, 0.13939375, 0.53175322, 0.68754078, 0.23754161,
          0.45106529, 0.1356445 ],
         [0.59655995, 0.65115982, 0.72767843, 0.85234067, 0.52833512,
          0.0981767 , 0.45166677],
         [0.44014486, 0.44810158, 0.79004692, 0.60239047, 0.54595107,
          0.78149435, 0.85564821],
         [0.10510811, 0.16368466, 0.39815677, 0.51708389, 0.71111921,
          0.35608862, 0.73065819],
         [0.36198063, 0.38257353, 0.15511632, 0.76413462, 0.54516488,
          0.64958072, 0.05417897],
         [0.34897044, 0.42899611, 0.27029773, 0.1114404 , 0.95430259,
          0.36595314, 0.81959748]],

        [[0.51348546, 0.73728077, 0.31380008, 0.62039746, 0.72558459,
          0.57220069, 0.85213679],
         [0.04663312, 0.77642579, 0.42425023, 0.10772509, 0.15422622,
          0.23879265, 0.3963137 ],
         [0.78968083, 0.30171326, 0.29162141, 0.36797

In [ ]:
col = im2col(x1, 5, 5, stride = 1, pad = 0)

In [ ]:
print(col.shape)

(9, 75)


In [ ]:
class Convolution:
    def __init__(self, W, b, stride = 1, pad = 0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = self.x.shape
        OH = int(1 + (H + 2*self.pad - FH)/self.stride)
        OW = int(1 + (W + 2*self.pad - FW)/self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad) #FW x FW * 채널수가 전체 개수와 같게 매트릭스 변형

        col_W = self.W.reshape(FN, -1)
        out = np.dot(col, col_W.T) +self.b
        out = out.reshape(N, OH, OW, -1).transpos(0,3,1,2)
        return out

# 풀링계층 구현하기

In [ ]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride = 1, pad =0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        OH = int(1 + (H+2*self.pad - self.pool_h)/self.stride)
        OW = int(1 + (W+2*self.pad - self.pool_w)/self.stride)

        col = im2col(x, pool_h, pool_w, self.stride, self.pad)
        col = col.reshape(-1, pool_h*pool_w)

        maxpooled =  max(col, axis = 1)

        out = maxpooled.reshape(N, OH, OW, C).transpose(0,3,1,2)

        return out
        

In [24]:
class SimpleConvNet:
    def __init__(self, input_dim = (1, 28, 28)
                        , conv_params = {"filter_num": 30, "filter_size" : 5, "pad" : 0, "stride" : 1 }
                        , hidden_size= 100, output_size =10, weight_std = 0.01):
        filter_num = conv_params["filter_num"]
        filter_size = conv_params["filter_size"]
        pad = conv_params["pad"]
        stride =conv_params["stride"]

        input_size = imput_dim[1]
        conv_output_size = int(1 + (input_size + 2*pad - filter_size)/stride)
        pool_output_size = (1, conv_output_size/2 * conv_output_size/2)

        params = {}

        params["W1"] = weight_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        params["b1"] = np.zeros(filter_num)
        params["W2"] = weight_std * np.random.randn(pool_output_size, hidden_size)
        params["b2"] = np.zeros(hidden_size)
        params["W3"] = weight_std * np.random.randn(hidden_size, output_size)
        params["b3"] = np.zeros(output_size)
    